# <center> **Data Wrangling** </center>
## API - Web Scraping

Realizaremos uma Análise Exploratória de Dados (EDA) para encontrar alguns padrões nos dados e determinar qual seria o rótulo para treinar modelos supervisionados.

No conjunto de dados, há vários casos diferentes em que o propulsor não pousou com sucesso. Às vezes, um pouso foi tentado, mas falhou devido a um acidente, por exemplo:
``True Ocean`` significa que o resultado da missão foi pouso com sucesso em uma região específica do oceano

``False Ocean`` significa que o resultado da missão foi pouso sem sucesso em uma região específica do oceano.

``True RTLS`` significa que o resultado da missão foi pouso com sucesso em uma plataforma de solo

``False RTLS`` significa que o resultado da missão foi pouso sem sucesso em uma plataforma de solo.

``True ASDS`` significa que o resultado da missão foi pouso com sucesso em um navio drone

``False ASDS`` significa que o resultado da missão foi pouso sem sucesso em um navio drone.

Converter esses resultados em rótulos para o target de treinamento, o 1 (Um) que significa que o propulsor pousou com sucesso, 0 (Zero) e não teve sucesso.

In [91]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [92]:
# Carregar o conjunto de dados CSV em pandas dataframe
df_spacex = pd.read_csv('/content/Db_bruto_SpaceX_falcon_09.csv')
print(df_spacex.shape)
df_spacex.head()

(90, 17)


,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857


In [93]:
# Identificar e calcular a porcentagem de valores ausentes em cada atributo
(df_spacex.isnull().sum()/len(df_spacex))*100

,0
FlightNumber,0.000000
Date,0.000000
BoosterVersion,0.000000
PayloadMass,0.000000
Orbit,0.000000
LaunchSite,0.000000
Outcome,0.000000
Flights,0.000000
GridFins,0.000000
Reused,0.000000


In [94]:
# Identificar as variaveis numericas e as categoricas
df_spacex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   FlightNumber    90 non-null     int64  
 1   Date            90 non-null     object 
 2   BoosterVersion  90 non-null     object 
 3   PayloadMass     90 non-null     float64
 4   Orbit           90 non-null     object 
 5   LaunchSite      90 non-null     object 
 6   Outcome         90 non-null     object 
 7   Flights         90 non-null     int64  
 8   GridFins        90 non-null     bool   
 9   Reused          90 non-null     bool   
 10  Legs            90 non-null     bool   
 11  LandingPad      64 non-null     object 
 12  Block           90 non-null     float64
 13  ReusedCount     90 non-null     int64  
 14  Serial          90 non-null     object 
 15  Longitude       90 non-null     float64
 16  Latitude        90 non-null     float64
dtypes: bool(3), float64(4), int64(3), obj

Cada lançamento tem como objetivo uma órbita específica, e aqui estão alguns tipos comuns de órbita



* <b>LEO</b>: Órbita terrestre baixa (LEO) é uma órbita centrada na Terra com uma altitude de 2.000 km (1.200 mi) ou menos (aproximadamente um terço do raio da Terra),[1] ou com pelo menos 11,25 períodos por dia (um período orbital de 128 minutos ou menos) e uma excentricidade menor que 0,25.[2] A maioria dos objetos artificiais no espaço sideral estão em LEO <a href='https://en.wikipedia.org/wiki/Low_Earth_orbit'>[1]</a>.

* <b>VLEO</b>: Órbitas Terrestres Muito Baixas (VLEO) podem ser definidas como órbitas com altitude média abaixo de 450 km. Operar nessas órbitas pode fornecer uma série de benefícios para naves espaciais de observação da Terra, pois a nave espacial opera mais perto da observação<a href='https://www.researchgate.net/publication/271499606_Very_Low_Earth_Orbit_mission_concepts_for_Earth_Observation_Benefits_and_challenges'>[2]</a>.


* <b>GTO</b> Uma órbita geossíncrona é uma órbita alta da Terra que permite que os satélites correspondam à rotação da Terra. Localizada a 22.236 milhas (35.786 quilômetros) acima do equador da Terra, esta posição é um ponto valioso para monitorar o clima, comunicações e vigilância. Como o satélite orbita na mesma velocidade em que a Terra está girando, o satélite parece permanecer no lugar em uma única longitude, embora possa derivar de norte a sul", escreveu a NASA em seu site Earth Observatory <a href="https://www.space.com/29222-geosynchronous-orbit.html" >[3] </a>.


* <b> SSO (or SO) </b>: É uma órbita heliossíncrona, também chamada de órbita heliossíncrona, é uma órbita quase polar ao redor de um planeta, na qual o satélite passa sobre qualquer ponto da superfície do planeta no mesmo tempo solar médio local <a href="https://en.wikipedia.org/wiki/Sun-synchronous_orbit">[4] <a>.   
    
    
* <b>ES-L1 </b>:Nos pontos de Lagrange, as forças gravitacionais dos dois grandes corpos se cancelam de tal forma que um pequeno objeto colocado em órbita ali está em equilíbrio em relação ao centro de massa dos grandes corpos. L1 é um desses pontos entre o sol e a terra <a href="https://en.wikipedia.org/wiki/Lagrange_point#L1_point">[5]</a> .
    
    
* <b>HEO</b> Uma órbita altamente elíptica é uma órbita elíptica com alta excentricidade, geralmente se referindo a uma órbita ao redor da Terra <a href="https://en.wikipedia.org/wiki/Highly_elliptical_orbit">[6] </a>.


* <b> ISS </b> Uma estação espacial modular (satélite artificial habitável) em órbita baixa da Terra. É um projeto colaborativo multinacional entre cinco agências espaciais participantes: NASA (Estados Unidos), Roscosmos (Rússia), JAXA (Japão), ESA (Europa) e CSA (Canadá)<a href="https://en.wikipedia.org/wiki/International_Space_Station"> [7] </a>


* <b> MEO </b> Órbitas geocêntricas variando em altitude de 2.000 km (1.200 mi) até logo abaixo da órbita geossíncrona a 35.786 quilômetros (22.236 mi). Também conhecida como órbita circular intermediária. Estas são "mais comumente a 20.200 quilômetros (12.600 mi), ou 20.650 quilômetros (12.830 mi), com um período orbital de 12 horas <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [8] </a>


* <b> HEO </b> Órbitas geocêntricas acima da altitude da órbita geossíncrona (35.786 km ou 22.236 mi) <a href="https://en.wikipedia.org/wiki/List_of_orbits"> [9] </a>

* <b> GEO </b> É uma órbita geossíncrona circular 35.786 quilômetros (22.236 milhas) acima do equador da Terra e seguindo a direção da rotação da Terra <a href="https://en.wikipedia.org/wiki/Geostationary_orbit"> [10] </a>


* <b> PO </b> É um tipo de satélite em que um satélite passa acima ou quase acima de ambos os polos do corpo orbitado (geralmente um planeta como a Terra) <a href="https://en.wikipedia.org/wiki/Polar_orbit"> [11] </a>

Alguns são mostrados no gráfico a seguir:

<center> <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/Orbits.png" width=650 height=450 /> </center>

Os dados contêm várias instalações de lançamento da Space X:

<a href='https://en.wikipedia.org/wiki/ist_of_Cape_Canaveral_and_Merritt_Island_launch_sites'>

* Cape Canaveral Space


</a> Launch Complex 40 <b> VAFB SLC 4E</b>, Vandenberg Air Force Base Space Launch Complex 4E <b>(SLC-4E)</b>, Kennedy Space Center Launch Complex 39A <b>KSC LC 39A</b>.

### A localização de cada lançamento é colocada na coluna <code>LaunchSite</code>

In [95]:
# Determinar o número de lançamentos em cada Site
df_spacex['LaunchSite'].value_counts()

,count
LaunchSite,
CCSFS SLC 40,55
KSC LC 39A,22
VAFB SLC 4E,13


In [96]:
#df_spacex.groupby(['LaunchSite'])['Flights'].sum()

In [97]:
# Calcular o número e a ocorrência de cada órbita
df_spacex['Orbit'].value_counts()

,count
Orbit,
GTO,27
ISS,21
VLEO,14
PO,9
LEO,7
SSO,5
MEO,3
ES-L1,1
HEO,1


In [98]:
# Contagem dos resultados de desfecho dos pousos da falcon 9
df_spacex['Outcome'].value_counts()

,count
Outcome,
True ASDS,41
None None,19
True RTLS,14
False ASDS,6
True Ocean,5
False Ocean,2
None ASDS,2
False RTLS,1


Aplica se neste momento, um tratamento a variavel ``Outcome``, pois, os dados, os quais identificam o desfecho do resultado do pouso do foguete Falcon 9, seram transformados em classes, com isso, esta variavel torna se categorica de duas classes.

As classes que perteceram a variavel Categorica ``Outcome`` são as seguintes:

* ``0`` **(Zero)** - Esta é a classe que identifica o desfecho do resultado pouso, como negativo, fracasso no pouso do foguete Falcon 9

* ``1`` **(Um)** - Esta é a classe que identifica o desfecho do resultado pouso, como positivo, sucesso no pouso do foguete Falcon 9

In [99]:
# Lista para armazenar as classes categorizadas
lista = []

# Lista com os atributos que identificam o resultado desfecho o pouso da falcon 9 como positivo/sucesso
lista_true = [
'True ASDS',
'True RTLS',
'True Ocean']

# Iterar linha por linha para identicar as classes e categorizar
for i in range(len(df_spacex)):
  if df_spacex['Outcome'][i] in lista_true:
     lista.append(1)
  else:
     lista.append(0)

# Inserindo uma nova variavel "Class" com as classes já categorizadas
df_spacex.insert(17, 'Class', lista, True)


In [100]:
# Verificando a nova variavel e a contagem dos atributos
df_spacex['Class'].value_counts()

,count
Class,
1,60
0,30


In [101]:
# Obtendo a média das classes
df_spacex['Class'].mean()

0.6666666666666666

In [102]:
# Salvando o Novo Dataset
df_spacex.to_csv('Db_categorizado_SpaceX_falcon_09.csv',
                 index=False,
                 index_label=False)

In [103]:
df_spacex.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0
